## RAG with LangChain, Ollama and FAISS Vector Store

In [ ]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True' # to avoid issue if FAISS and Chroma exists in same environment
warnings.filterwarnings("ignore")

load_dotenv()

In [ ]:
os.environ['LANGCHAIN_PROJECT']

### Document Loader 

### single pdf loading using PyMuPDF

In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
loader = PyMuPDFLoader("./rag_dataset/gym_supplements/Analysis_of_ActualFitnessSupplement.pdf")
docs = loader.load()

In [ ]:
docs

In [ ]:
doc = docs[0]
print(doc.page_content)

### Multiple pdfs loading

In [ ]:
import os

pdfs = []
for root, dirs, files in os.walk('rag_dataset'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))
print(pdfs)

In [ ]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()

    docs.extend(pages)

print(len(docs))  # 64 pages of the documents

In [ ]:
docs

### Document Chunking

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

chunks = text_splitter.split_documents(docs)


In [ ]:
len(docs), len(chunks)   # comparing.. 

In [ ]:
len(docs[0].page_content), len(chunks[0].page_content)

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

len(encoding.encode(docs[0].page_content)), len(encoding.encode(chunks[0].page_content))

### Document Vector Embedding

In [6]:


from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore



In [7]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url="http://localhost:11434")

single_vector = embeddings.embed_query("this is some text data")

In [ ]:
print(single_vector)

In [ ]:
len(single_vector)

In [8]:
index = faiss.IndexFlatL2(len(single_vector))
index.ntotal, index.d


(0, 768)

In [9]:
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [ ]:
len(chunks)

In [ ]:


ids = vector_store.add_documents(documents=chunks)



In [ ]:
vector_store.index_to_docstore_id
len(ids)

In [10]:
# store vector database
db_name = "health_supplements"
# vector_store.save_local(db_name)

# load vector database
new_vector_store = FAISS.load_local(db_name, 
                                    embeddings=embeddings, 
                                    allow_dangerous_deserialization=True)
len(new_vector_store.index_to_docstore_id)


311

In [ ]:

question = "what is used to gain muscle mass?"
docs = vector_store.search(query=question, search_type='similarity')

for doc in docs:
    print(doc.page_content)
    print("\n\n")


### retriever

In [11]:

retriever = vector_store.as_retriever(search_type="mmr", search_kwargs = {'k': 3, 
                                                                          'fetch_k': 100,
                                                                          'lambda_mult': 1})



In [ ]:
docs = retriever.invoke(question)

for doc in docs:
    print(doc.page_content)
    print("\n\n")

In [ ]:
question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
# question = "what are the benefits of BCAA supplements?"
docs = retriever.invoke(question)
for doc in docs:
    print(doc.page_content)
    print("\n\n")


### RAG with LLAMA 3.2 on OLLAMA

In [12]:


from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_ollama import ChatOllama



In [ ]:
model = ChatOllama(model="llama3.2:1b", base_url="http://localhost:11434")

model.invoke("hi")

In [ ]:
prompt = hub.pull("rlm/rag-prompt")

In [ ]:
prompt

In [ ]:
prompt = """
    You are an assistant for question-answering tasks based on gym and health supplements. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Answer in bullet points. Make sure your answer is relevant to the question and it is answered from the context only.
    Question: {question} 
    Context: {context} 
    Answer:
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt)

In [ ]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# print(format_docs(docs))


In [ ]:
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
# question = "what is used to gain muscle mass?"
# question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
question = "what are the benefits of BCAA supplements?"

output = rag_chain.invoke(question)
print(output)

In [13]:
from langchain import hub #TO PULL RAG PROMPTS
from langchain_core.output_parsers import StrOutputParser # GETTING FINAL OUT AS STRING
from langchain_core.runnables import RunnablePassthrough #parse question and context directly to LLM
from langchain_core.prompts import ChatPromptTemplate #to pass prompt with context (chunk of data)

from langchain_ollama import ChatOllama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [14]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])
# print(format_docs(docs))

model = ChatOllama(
    model="llama3.2:1b",
    base_url="http://localhost:11434",
    streaming=True,  # Enable streaming
    callbacks=[StreamingStdOutCallbackHandler()],  # Print tokens as they are generated
)

prompt = """
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Make sure your answer is relevant to the question and it is answered from the context only. Only answer in English.
    Question: {question} 
    Context: {context} 
    Answer:
"""

prompt = ChatPromptTemplate.from_template(prompt)
rag_chain = (
    {"context": retriever|format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [15]:
# question = "what is used to gain muscle mass?"
# question = "what is used to reduce weight?"
# question = "what are side effects of supplements?"
# question = "what are the benefits of supplements?"
question = "what are the benefits of BCAA supplements?"

output = rag_chain.invoke(question)
print(output)

BCAAs (Brachyury, Branchial Archura, and Cysteine) are amino acids that play a crucial role in muscle growth, recovery, and maintenance. Here are some benefits of taking BCAA supplements:

1. **Muscle Recovery**: BCAAs can help reduce muscle soreness (DOMS) and speed up the recovery process after intense exercise, making them ideal for athletes and fitness enthusiasts.
2. **Reduced Muscle Damage**: By reducing muscle damage, BCAAs may help prevent further inflammation and promote faster healing of muscle tissues.
3. **Increased Muscle Protein Synthesis**: BCAAs can stimulate muscle protein synthesis, which is essential for building and repairing muscle tissue.
4. **Improved Athletic Performance**: Some studies suggest that taking BCAA supplements before or during exercise may improve athletic performance by increasing endurance, strength, and power.
5. **Antioxidant Properties**: BCAAs have antioxidant properties, which can help protect muscles from oxidative stress and damage caused b